# Assignment 2 - ST449 Deep Learning and Artificial Intelligence

In [46]:
import numpy as np
from tqdm import tqdm
import itertools
from itertools import product
from environments import *

# set numpy pring options
np.set_printoptions(precision=2)

## Part 1
### Question 1.1

I provide two solutions to this question. One is based on reducing the problem to a Markov Reward Process (a) and the other one is based on iterative policy evaluation (b).

*Note: The multi-line equations do not render properly in GitHub, please read the HTML file for the correct solution. This is only relevant for this part.*

#### a) Solution based on reduction to a Markov Reward Process (MRP)

Given complete knowledge about the dynamics and rewards of the environment, a finite Markov Reward Process (MRP) can be solved in closed form by reforming the Bellman equation for an MRP.

$$
V = R+\gamma PV\\\\
V-\gamma PV = R\\\\
(I-\gamma P)V = R\\\\
V = (I-\gamma P)^{-1}R
$$

An MRP is defined by the tuple $(S,P,R,\gamma)$ consisting of the set of states $S$, the transition matrix $P$, the reward function $R$, and a discount factor $\gamma$. There are no actions in an MRP. An MDP, by contrast, is defined by $(S,P,R,A,\gamma)$, where the additional $A$ represents a set of actions. 

An important fact is that **given a policy, an MDP reduces to an MRP**. In other words, we can represent an MDP as an MRP with the tuple $(S, R^{\pi}, P^{\pi}, \gamma)$. As a result, the Markov Decision Process (MDP) can be solved by reduction to an MRP. That is, $V^{\pi}$ can be calculated in closed form as follows.

$$
V^{\pi} = R^{\pi}+\gamma P^{\pi}V^{\pi}\\\\
...\\\\
V^{\pi} = (I-\gamma P^{\pi})^{-1}R^{\pi}
$$

where $R^{\pi}=R^{\pi}(s) = \sum_a\pi(a|s)R(s,a)$ is a vector of expected immediate rewards under policy $\pi$ and $P^{\pi}=P^{\pi}(s'|s) = \sum_a\pi(a|s)P(s'|s,a)$ is the matrix of transition probabilities under policy $\pi$. By defining $R^{\pi}$ and $P^{\pi}$, we reduce the MDP to an MRP, which can be solved analytically.

Note that $V^{\pi} = R^{\pi}+\gamma P^{\pi}V^{\pi}$ is the Bellman equation in matrix notation, which is equivalent to $V_\pi(s)=\sum_a\pi(a|s)\sum_{s',r} p(s'r|s,a)(r+\gamma V_\pi(s')$ for all $s$.

##### Deriving all policies
The goal of this exercise is to evaluate all deterministic policies. In a finite MDP, there are $|A|^{|S|}$ unique deterministic policies, which means that in this problem we have $2^5=32$ policies.

I represent the as vectors of length 7, including the two terminal states. Action $LOW$ is represented by 0 and $HIGH$ by 1. Note that the action in the terminal states does not matter because the agent does not get to act once she reaches that state. Therefore, most often I will only show the five non-terminal states.

In [2]:
HIGH = 1
LOW = 0

# list all polices
policies = [list(i) for i in product(range(2), repeat=5)]
policies = np.array([[0] + policy + [0] for policy in policies])
print('# of polices:', len(policies))
print(policies[:, 1:6])

# of polices: 32
[[0 0 0 0 0]
 [0 0 0 0 1]
 [0 0 0 1 0]
 [0 0 0 1 1]
 [0 0 1 0 0]
 [0 0 1 0 1]
 [0 0 1 1 0]
 [0 0 1 1 1]
 [0 1 0 0 0]
 [0 1 0 0 1]
 [0 1 0 1 0]
 [0 1 0 1 1]
 [0 1 1 0 0]
 [0 1 1 0 1]
 [0 1 1 1 0]
 [0 1 1 1 1]
 [1 0 0 0 0]
 [1 0 0 0 1]
 [1 0 0 1 0]
 [1 0 0 1 1]
 [1 0 1 0 0]
 [1 0 1 0 1]
 [1 0 1 1 0]
 [1 0 1 1 1]
 [1 1 0 0 0]
 [1 1 0 0 1]
 [1 1 0 1 0]
 [1 1 0 1 1]
 [1 1 1 0 0]
 [1 1 1 0 1]
 [1 1 1 1 0]
 [1 1 1 1 1]]


In addition to the 32 policies, this solution also requires all 32 transition matrices $P^{\pi}$ and reward vectors $R^{\pi}$. Both can be easily inferred from the description of the game (or the environment essentially), although it is not that straightforward to construct them.

Importantly, I compute the transition matrices and reward vectors not only because I want to present my closed form MRP solution here. As will become clear later, the iterative policy evaluation algorithm can also be elegantly implemented in a vectorized fashion using these transition matrices and reward vectors.

In [3]:
transition_matrices = []

Tsa = np.zeros((2,7,7))

for i in range(1,6):
    Tsa[0, i, i-1] = 0.55
    Tsa[0, i, i+1] = 0.45
    
for i in range(1,6):
    Tsa[1, i, i-1] = 0.45
    Tsa[1, i, i+1] = 0.55

# generate the transition matrix for each policy
for policy in policies:
    P = Tsa[policy, range(7)]
    transition_matrices.append(P)

print(f"Example: Transition matrix for policy {policies[0]}, or 'always choose LOW':\n")
print(transition_matrices[0])

Example: Transition matrix for policy [0 0 0 0 0 0 0], or 'always choose LOW':

[[0.   0.   0.   0.   0.   0.   0.  ]
 [0.55 0.   0.45 0.   0.   0.   0.  ]
 [0.   0.55 0.   0.45 0.   0.   0.  ]
 [0.   0.   0.55 0.   0.45 0.   0.  ]
 [0.   0.   0.   0.55 0.   0.45 0.  ]
 [0.   0.   0.   0.   0.55 0.   0.45]
 [0.   0.   0.   0.   0.   0.   0.  ]]


In [4]:
# generate the vector of expected immediate rewards for each policy
reward_vectors = []

Rsa = np.zeros((7,2))

for i in range(1,6):
    Rsa[1:6, 0] = -10
    Rsa[1:6, 1] = -50

    Rsa[5, 0] = -10*0.55 + 0.45*990
    Rsa[5, 1] = -50*0.45 + 0.55*950

for policy in policies:
    reward_vectors.append(Rsa[range(7), policy])
    
print("Example: Vector of expected immediate rewards for policy 'always choose LOW':")
print(reward_vectors[0])

Example: Vector of expected immediate rewards for policy 'always choose LOW':
[  0. -10. -10. -10. -10. 440.   0.]


In the code cell below, I show that a closed form solution for a policy (for example "always choose LOW") can indeed be obtained by calculating

$$
V^{\pi} = (I-\gamma P^{\pi})^{-1}R^{\pi}
$$

In [5]:
# solve MDP in closed form
P = transition_matrices[0]
R = reward_vectors[0]

V = np.linalg.inv(np.eye(7)-P).dot(R)
print('state values:', V)

state values: [  0.    52.37 138.6  266.21 444.41 684.42   0.  ]


<br>

#### b) Iterative Policy Evaluation

Solving an MRP involves computing a matrix inverse, which has computational complexity $O(N^3)$ and is therefore unfeasible if the state and/or action space are large. In that case, iterative policy evaluation can be used.

As already mentioned, the transition matrices and reward vectors were not only useful for obtaining a closed form solution, they also vastly simplify the implementation of the _iterative policy evaluation_ algorithm. That is, instead of for loops, plain and elegant matrix algebra can be used to update the state values. More specifically, $V^{\pi}$ can be obtained by repeatedly applying the Bellman backup to some initial $V^{0}$ until a convergence criterion is met.

In mathematical terms, the update can be written as follows.

$$
V_{\pi}^{t+1} = R^{\pi} + P^{\pi}V_{\pi}^{t} = BV_{\pi}^{t}
$$

where $B$ represents the **Bellman backup** (and not a matrix multiplication). Applying this backup repeatedly can be seen as 

$$
V_{\pi} = BB...BV_{\pi}^{0}
$$

Even though the matrix algebra makes this solution more elegant compared to a solution using for loops, there is a potential downside: one Bellman backup in matrix form essentially represents a full sweep over all states. In contrast, by using a for loop, the state value updates can be done in-place, which may speed up convergence.

Below is an estimate for the state values of the policy "always choose LOW". As can be seen, the state values are equivalent to the ones obtained in closed form above.

In [6]:
V = np.zeros(7)

deltaV = 1
# repeatedly apply bellman update
while deltaV >= 1e-6:
    V_old = V
    V = R + P.dot(V)
    deltaV = np.linalg.norm(V-V_old)

print('state values:', V)

state values: [  0.    52.37 138.6  266.21 444.41 684.42   0.  ]


<br>

##### Estimate all 32 policies
To get state value estimates for all 32 policies, we can apply this iterative policy evaluation algorithm to each policy $\pi$. Finally, we sort the policies by their value in the initial state.

In [7]:
# collect the state values for each policy
state_values = []

# run policy evlauation for each policy
for pi, T, R in zip(policies, transition_matrices, reward_vectors):
    
    V = np.zeros(7)

    deltaV = 1
    # repeatedly apply bellman update
    while deltaV >= 1e-6:
        V_old = V
        V = R + T.dot(V)
        deltaV = np.linalg.norm(V-V_old)
        
    state_values.append(V)

In [8]:
# get the max of state 3 (start state)
best_policy_index = np.array([x[3] for x in state_values]).argmax()

# best policy is LOW whenever d<=0 and HIGH for d>0
print('optimal policy:', policies[best_policy_index, 1:6])
print('state values:', state_values[best_policy_index])

optimal policy: [0 0 0 1 1]
state values: [  0.    56.52 147.83 281.65 467.43 710.35   0.  ]


##### Discussion of the Results
The optimal deterministic policy for starting at the initial state chooses action $HIGH$ whenever the player has a lead over her opponent, and $LOW$ otherwise. 

Most likely, this is because the agent is driven by the positive reward in the states close the winning state while in states that are closer to loosing, the agent does not want to occur the high cost of action $HIGH$ when the likelihood of losing is quite high in any way. Generally, among the top 10 policies, most confirm this pattern of choosing high in states close to a win and low in states closer to a loss.

In fact, this policy is not only the optimal fixed policy when starting at the initial state but also the optimal policy overall. To recap, an optimal policy $\pi^*$ is defined as a policy whose state values are greater than or equal to those of all other policies. This is the case for policy $[0,0,0,1,1]$.

In addition, it is also worth mentioning that an optimal policy is always a deterministic policy (that is, unless there are states in which both actions are equally good (have the same action values), in which case any policy that stochastically chooses among them is optimal too). This is due to the fact that the optimal action value function $Q^*$ is unique (it is the unique solution to the Bellman equation). As a result of our exhaustive search for the best (deterministic) policy, we can be sure that $[0,0,0,1,1]$ is indeed the optimal policy for that game. In particular, no stochastic policy can be as good (and we already know that all deterministic policies are worse).

Below is the ranking of all 32 polices from best to worst (based on the initial state). Intriguingly, the two best polices are the opposites of the two worst policies, but this pattern is not consistent throughout the best/worst 16 policies.

In [9]:
# sort and print all polcies
np.set_printoptions(suppress=True)
state_values = np.array(state_values)
ranking = reversed(np.argsort(state_values[:, 1]).tolist())
print("Policies ranked from best to worst:")
print(policies[list(ranking), 1:6])

Policies ranked from best to worst:
[[0 0 0 1 1]
 [0 0 0 0 1]
 [0 0 0 1 0]
 [0 0 0 0 0]
 [0 0 1 1 1]
 [0 0 1 0 1]
 [0 0 1 1 0]
 [0 0 1 0 0]
 [0 1 0 1 1]
 [0 1 0 0 1]
 [0 1 1 1 1]
 [0 1 0 1 0]
 [0 1 0 0 0]
 [0 1 1 0 1]
 [0 1 1 1 0]
 [0 1 1 0 0]
 [1 0 0 1 1]
 [1 0 0 0 1]
 [1 0 0 1 0]
 [1 0 1 1 1]
 [1 0 0 0 0]
 [1 0 1 0 1]
 [1 0 1 1 0]
 [1 0 1 0 0]
 [1 1 0 1 1]
 [1 1 1 1 1]
 [1 1 0 0 1]
 [1 1 0 1 0]
 [1 1 1 0 1]
 [1 1 0 0 0]
 [1 1 1 1 0]
 [1 1 1 0 0]]


### Question 1.2 - Off-policy Monte Carlo Prediction (Policy Evaluation)
From the instructions, it is not quite clear whether the evaluation policy should be $[1,0,0,0,0]$ or $[1,1,1,1,0]$, depending on which encoding of the states is implied. I went with the latter.

In [2]:
# define the two policies
def evaluation_policy(state):
    if state <= 4:
        return HIGH
    else:
        return LOW
    
def behavior_policy(state):
    return np.random.choice(2)

# show eval policy,
print('Evaluation policy:', [evaluation_policy(state) for state in range(1,6)])

Evaluation policy: [1, 1, 1, 1, 0]


In [29]:
env = Game()

# intialize
Q = np.zeros((7,2))
C = np.zeros((7,2))
gamma = 1

episodes = 500000
for episode in tqdm(range(episodes)):
    
    history = []
    state = env.reset()
    
    # generate an episode
    done = False
    while not done:

        action = behavior_policy(state)
        next_state, reward, done = env.step(action)
        
        history.append((state, action, reward))
        
        state = next_state
        
    G = 0
    W = 1

    # do the MC updates
    for state, action, reward in reversed(history):
    
        G = gamma*G + reward
        C[state, action] += W
        Q[state, action] += (W/C[state, action]) * (G-Q[state, action])
                
        if evaluation_policy(state) == action:
            W *= 1/0.5
        else:
            W = 0
            break

100%|███████████████████████████████████████████████████████████████████████| 500000/500000 [00:30<00:00, 16397.04it/s]


In [30]:
## MC prediction results, 500K episodes
print('action values:')
print(Q)
print('suggested policy:', np.argmax(Q, axis=1)[1:6])

action values:
[[  0.     0.  ]
 [ 56.8   18.9 ]
 [137.7  147.29]
 [327.86 339.01]
 [498.38 574.88]
 [806.3  718.13]
 [  0.     0.  ]]
suggested policy: [0 1 1 1 0]


In [18]:
## MC prediction results, 500K episodes
print('action values:')
print(Q)
print('suggested policy:', np.argmax(Q, axis=1)[1:6])

action values:
[[  0.     0.  ]
 [ 54.96   1.13]
 [111.17  62.79]
 [298.94 159.11]
 [426.89 448.48]
 [658.42 676.95]
 [  0.     0.  ]]
suggested policy: [0 0 0 1 1]


To begin with, the Monte Carlo results were very unstable, which is why it is difficult to interpret this very result. Above are two runs of 500K episodes. The action values are more different that expected because in theory, Monte Carlo estimates are guaranteed to converge to the true values (see 1.3 for further details about that).

Generally speaking, because the policy inferred from the action values is different from the evaluation policy, we can say that it is not the optimal policy (assuming that the action values indeed converged).

In other words, the action values suggest a policy that is different from the evaluation policy. By the _policy improvement theorem_, if we act greedily with respect to these action values, we can improve upon the original policy, i.e. the evaluation policy. This is because

$$
q_\pi(s, \pi'(s)) \ge v_\pi(s)
$$

where $\pi' = \arg\max_a q_\pi(s,a)$ is always true. This is the general idea of _policy iteration_.

Indeed, when compared to the ranking of policies from part 1.1, we can see that the newly suggested policies (in both runs) are indeed better than the evaluation policy (which is the second worst deterministic policy). The second run actually suggests the optimal policy. However, as already mentioned, this is not always the case.

### Question 1.3 - Off-policy Monte Carlo Control

In [31]:
def greedy_policy(qvalues):
    return np.argmax(qvalues)

In [32]:
# initialize the environment
env = Game()

# intialize
Q = np.zeros((7,2))
C = np.zeros((7,2))
gamma = 1

episodes = 500000
for episode in tqdm(range(episodes)):
          
    history = []
    state = env.reset()
    
    # generate an episode
    done = False
    while not done:

        action = behavior_policy(state)
        next_state, reward, done = env.step(action)
        
        history.append((state, action, reward))
        
        state = next_state
        
    G = 0
    W = 1
        
    # perform the MC updates
    for state, action, reward in reversed(history):
        
        G = gamma*G + reward
        C[state, action] += W
        Q[state, action] += (W/C[state, action]) * (G-Q[state, action])
        
        max_action = greedy_policy(Q[state])
                
        if action != max_action:
            break
        else:
            W /= 0.5

100%|███████████████████████████████████████████████████████████████████████| 500000/500000 [00:34<00:00, 14368.65it/s]


In [33]:
## MC control results, 500K episodes
print(Q)
print('suggested optimal policy:', np.argmax(Q, axis=-1)[1:6])

[[  0.     0.  ]
 [  5.56  18.46]
 [129.66 155.63]
 [358.16 364.81]
 [580.54 547.37]
 [787.41 795.  ]
 [  0.     0.  ]]
suggested optimal policy: [1 1 1 0 1]


In [60]:
## MC control results, 500K episodes
print(Q)
print('suggested optimal policy:', np.argmax(Q, axis=-1)[1:6])

[[  0.     0.  ]
 [ 13.94  28.64]
 [199.6  134.24]
 [411.6  317.52]
 [524.96 520.96]
 [690.15 750.18]
 [  0.     0.  ]]
suggested optimal policy: [1 0 0 0 1]


As before, the action value estimates are quite unstable and the resulting optimal policy changes accordingly from run to run. Generally, we would expect the algorithm to find the optimal policy from part 1.1, which is $[0,0,0,1,1]$.

A potential reason could be that the number of episodes is too low. In fact, off-policy Monto Carlo is very inefficient using a random behavior policy. The probability for only one single action value update is $0.5$. The probability of two updates is $0.5^2=0.25$, for three it is $0.5^3=0.125$, and so on. Note that the updates occur in reversed order, so only the last (few) state-action values receive updates (if any). Therefore, even though some episodes last more than ten rounds, the number of updates is typically much lower. 

To highlight this problem: say the agent reaches state 5 (just before winning the game) but ends up in state 0 (the losing state). In this this episode, state 5 will most likely not receive an update, but only the state-action values close to 0. Of course, this problem also applies to the Monte Carlo policy evaluation (part 1.2) and may also explain the instability of the action value estimates there.

## Part 2
### Question 2.1 - SARSA
In this and the next part (Q-learning), I use a step-size of $\alpha=0.001$.

In [84]:
# define the behavior policy
def eps_greedy_policy(qsa, epsilon=0.1):
    if np.random.binomial(1, epsilon) == 1:
        return np.random.choice(2) # only two actions
    else:
        return np.random.choice([action_ for action_, value_ in enumerate(qsa) if value_ == np.max(qsa)])

# sarsa update function
def sarsa(qsa, next_qsa, r, alpha=0.001, gamma=1.0):
    return qsa + alpha * (r + gamma * next_qsa - qsa)

In [85]:
# initialize action values
Q = np.zeros((7,2))

episodes = 500000
for episode in tqdm(range(episodes)):
        
    # reset the env and get fist state
    state = env.reset()
    
    # for sarsa, first action before loop
    action = eps_greedy_policy(Q[state])
    
    done = False
    while not done:
        
        # step environment
        next_state, reward, done = env.step(action)
        
        # get sarsa next action, on-policy
        next_action = eps_greedy_policy(Q[next_state])
        
        # sarsa update
        Q[state, action] = sarsa(Q[state, action], Q[next_state, next_action], reward)
        
        # update state and action
        state = next_state
        action = next_action

100%|████████████████████████████████████████████████████████████████████████| 500000/500000 [02:10<00:00, 3842.01it/s]


In [70]:
## SARSA results 500K episodes, alpha 0.001
print(Q)
np.argmax(Q, axis=1)

[[  0.           0.        ]
 [ 55.33392574  30.51102454]
 [141.55013044 127.78302871]
 [281.33146712 267.08369879]
 [453.51996653 471.27609031]
 [695.67626223 720.29911683]
 [  0.           0.        ]]


array([0, 0, 0, 0, 1, 1, 0], dtype=int64)

The Q-values learned by SARSA suggest a policy that chooses action $HIGH$ whenever she has a lead over her opponent, otherwise $LOW$. This coincides with the optimal policy found through exhaustive search. 

It is not always the case that SARSA indeed finds the optimal policy as it essentially solves a slightly different problem: it searches not for the "globally" optimal policy but for the optimal policy among all $\epsilon$-greedy (or $\epsilon$-soft) policies. This is because it is an on-policy algorithm which has to balance the exploration-exploitation tradeoff. Sometimes, as a result, SARSA leads to a policy that stays further away from states with a high negative reward because it cannot safely move in the close by area. This game does not have any such states, however.

Another intriguing observation about the optimal policy is the following: I already mentioned how the agent is driven by the positive reward in states close to a win while in states close to a loss, the high effort is just "not worth it" since a loss is quite likely in any way. Put differently, the agent avoids the additional costs incurred by taking action $HIGH$ in favor of a quick loss which does not incur any additional negative reward. If, say, a loss was associated with a negative reward of 1000, the agent would be incentivized to avoid the losing state. We would then expect the agent to choose action $HIGH$ in states close to a loss in order to escape this area as quickly as possible.

### Question 2.2 - Q-Learning

In [73]:
def q_learning(qsa, next_qs, r, alpha=0.001, gamma=1.0):
    return qsa + alpha * (r + gamma * np.max(next_qs) - qsa)

In [77]:
Q = np.zeros((7,2))

episodes = 500000
for episode in tqdm(range(episodes)):
    
    state = env.reset()
    
    done = False
    while not done:
        
        # act according to behavior policy
        action = eps_greedy_policy(Q[state])
        
        # step environment
        next_state, reward, done = env.step(action)
        
        # qlearning update
        Q[state, action] = q_learning(Q[state, action], Q[next_state], reward)
        
        # set state to next state
        state = next_state

100%|████████████████████████████████████████████████████████████████████████| 500000/500000 [02:15<00:00, 3690.14it/s]


In [78]:
# Q-learning results, 500K episodes, alpha 0.001
print(Q)
np.argmax(Q, axis=1)

[[  0.           0.        ]
 [ 57.26453492  29.49739819]
 [152.91018034 133.1679582 ]
 [283.55266974 269.90749708]
 [460.01745826 468.03697322]
 [691.62559894 717.26099413]
 [  0.           0.        ]]


array([0, 0, 0, 0, 1, 1, 0], dtype=int64)

Like SARSA, the action values obtained though Q-learning suggest a policy that chooses action $HIGH$ whenever she has a lead over her opponent, otherwise $LOW$. This is the optimal policy, which is the expected result of this off-policy method. 

Indeed, unlike SARSA, Q-learning does not have to content with an approximate optimal policy. This is because the exploration-exploitation tradeoff is taken care of by using two policies - a _behavior policy_ and an _evaluation policy_. While this is more general and powerful, it is also more difficult to train. In fact, on-policy control is actually a special case of off-policy control where the behavior policy is also the evaluation policy.

### Question 2.3 - TD($\lambda$)

>**Note**: Algorithms using an eligibility trace should set the trace to 0 in each round, otherwise rewards obtained in one episode propagate to the next, which is unwanted. The pseudocode suggested in the lecture slides does not do this. Most likely, this mistake originated in the first edition of Sutton and Barto's Reinforcement Learning book, see [here](https://stackoverflow.com/questions/29904270/eligibility-trace-reinitialization-between-episodes-in-sarsa-lambda-implementati).

In [34]:
def random_policy():
    return np.random.choice(2)

In [15]:
env = Game()

V = np.zeros(7)

# set some variables
step_size = 0.0001
gamma = 1
lambda_ = 0.9

episodes = 500000
for episode in tqdm(range(episodes)):
    
    e = np.zeros(7) 
    state = env.reset()
    
    done = False
    while not done:
        
        # act according to behavior policy
        action = random_policy()
        
        # step environment
        next_state, reward, done = env.step(action)
        
        # compute td error and perfrom updates
        td_error = reward + gamma * V[next_state] - V[state]
        e[state] += 1
            
        V += step_size * e * td_error
        e *= gamma * lambda_
        
        # set state to next state
        state = next_state

100%|████████████████████████████████████████████████████████████████████████| 500000/500000 [01:46<00:00, 4716.59it/s]


In [16]:
# td lambda results, 500K episodes
V

array([  0.        ,  15.16744125,  89.18443075, 224.24735204,
       421.93389927, 680.03555275,   0.        ])

Judging by the policy value of the starting state, we can see that the random policy would rank somewhere in the middle of all deterministic policies. In other words, a random policy would do better than roughly half of all deterministic policies, which is quite unexpected.

## Part 3 - SARSA($\lambda$)

In contrast to the previous setup, there are now $7x11$ states due to the fact that there are $7$ different scores (from 0 to 6) and $11$ different energy levels (from 0 to 10). In addition, the agent now faces a constraint on the action choice when the energy level is 0. The most natural place to implement that constraint is the policy, which can be seen as the agent's strategy for taking actions. In any state with energy level 0, she simply has no other choice than to select action $LOW$.

In [36]:
def constr_eps_greedy_policy(qs, energy, epsilon=0.1):
    
    if energy == 0:
        return LOW
    
    if np.random.binomial(1, epsilon) == 1:
        return np.random.choice(2) # only two actions
    
    else:
        return np.random.choice([action_ for action_, value_ in enumerate(qs) if value_ == np.max(qs)])
        # or np.argmax(qs)

In [93]:
env = EnergyGame()

Q = np.zeros((7,11,2))
# C = np.zeros_like(Q)

# set some variables
step_size = 0.001
gamma = 0.9
lambda_ = 0.9

episodes = 500000
for episode in tqdm(range(episodes)):
    
    e = np.zeros_like(Q)
    state = env.reset()
    
    # for sarsa, first action before loop
    action = constr_eps_greedy_policy(Q[state], state[1])
    
    done = False
    while not done:
        
        # step environment
        next_state, reward, done = env.step(action)
        
        # get sarsa next action, on-policy
        next_action = constr_eps_greedy_policy(Q[next_state], next_state[1])
        
        # compute td error
        td_error = reward + gamma * Q[next_state][next_action] - Q[state][action]
        e[state][action] += 1
        
        # update Q and eligibility trace
        Q += step_size * e * td_error
        e *= gamma * lambda_
        
        # C[state][action] +=1
        
        # update state and action
        state = next_state
        action = next_action

100%|████████████████████████████████████████████████████████████████████████| 500000/500000 [03:42<00:00, 2245.76it/s]


In [94]:
# SARSA lambda results, 500K episodes
print('State-action values after 500K episodes (dimensions are score, energy, action):')
print(Q[1:6])
print('\nSuggested policy:')
print(np.argmax(Q, axis=-1))

State-action values after 500K episodes (dimensions are score, energy, action):
[[[ 47.49   0.  ]
  [ 28.96   2.53]
  [ 32.24  65.86]
  [ 55.99  69.49]
  [ 56.26  76.89]
  [ 73.06  48.7 ]
  [ 66.42  89.95]
  [ 20.32  80.62]
  [ 71.12  96.2 ]
  [ 55.7   90.25]
  [ 70.99  90.08]]

 [[113.05   0.  ]
  [ 62.06 125.85]
  [140.52 107.08]
  [154.71 137.37]
  [129.72 168.5 ]
  [151.16 178.75]
  [132.27 169.69]
  [157.49 193.85]
  [188.67 155.6 ]
  [167.43 194.32]
  [169.73 185.26]]

 [[205.74   0.  ]
  [140.44 238.73]
  [155.2  255.72]
  [226.75 286.  ]
  [252.73 296.45]
  [214.78 282.5 ]
  [265.5  323.84]
  [287.68 228.42]
  [275.38 324.76]
  [273.95 300.84]
  [287.74 329.57]]

 [[361.45   0.  ]
  [402.86 175.28]
  [327.26 452.34]
  [409.88 464.8 ]
  [440.17 397.87]
  [439.93 487.17]
  [455.95 430.39]
  [439.38 498.49]
  [451.81 468.86]
  [440.99 508.49]
  [454.97 468.96]]

 [[639.59   0.  ]
  [387.57 710.54]
  [150.55 705.73]
  [666.35 482.7 ]
  [622.55 741.93]
  [223.28 729.32]
  [683.9  74

To begin with, as was the case with Monte Carlo in parts 1.2 and 1.3, the state-action values do not fully converge with 500K episodes. As a result, the values as well as the resulting optimal policy can change slightly from run to run.

First things first, the state-action values increase with both the score and the energy level, which is expected. The optimal policy appears to suggest action $HIGH$ in most states. Especially if the energy level is high (10 or close to 10), the agent can safely take the energy loss since most episodes only last a few rounds, no matter the current state. In fact, the energy budget is quite large for the game such that states with low energy get visited only rarely, which leads to a higher variance in the estimates for those states.

Due to the constraint of having to choose action $LOW$ if the energy is $0$, we would expect the agent to avoid those states. Especially if the score is low too. In other words, sometimes it makes more sense to choose action low in order to save energy for when it is needed most, which is when the agent is close to the losing state. 

That being said, learning the optimal policy given these parameters is not easy. First of all, the results seem to be influenced by the very first action taken in each state. This is because of the following:

Say the state-action values are initialized with 0. Early on, each time an action value is updated, it will be a positive (or 0) update and that action will therefore be chosen with a high probability (i.e. $1-\epsilon$) again in the future. Since all updates are small and incremental, it is likely that the state-action values who got updated first "run away" from those that did not get updated early in the game. As a result, they cannot catch up. Of course, eventually all state-action values should converge. However, the number of episodes may simply not be large enough. This is especially problematic for states that are not visited often, which is true for states with a low energy level, and even more so for states with not only a low energy level but also a low score.

A potential solution to this problem would be to use optimistic initial values. In essence, the problem I just described due to insufficient exploration, especially in the early phase of the game. Optimistic initial values encourage exploration.

Below is the optimal policy found after 2 Mio episodes instead of 500K. We can see that action $HIGH$ is even more prevalent except for a few states in which the energy is at level 1.

In [92]:
# SARSA lambda results, 2 Mio episodes
#print(Q[1:6])
print('Suggested policy after 2 Mio episodes:')
print(np.argmax(Q, axis=-1))

Suggested policy after 2 Mio episodes:
[[0 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 1 1 1 1 1 1 1 1]
 [0 0 1 1 1 1 1 1 1 1 1]
 [0 0 1 1 1 1 1 1 1 1 1]
 [0 1 1 1 1 1 1 1 1 1 1]
 [0 0 1 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0]]


## Part 4

#### 4.1 Winning probability of player $X$ when $x=1$ and any value of $y$.

Player $X$ cannot afford to lose even a single round of the game. She must win all rounds, which is until player $Y$ has lost all her tokens. The probability of winning each round of the game is $0.5$. Therefore,

$$
P(win|x=1, y) = 0.5^y 
$$

Alternatively, the probability can also be defined in a recursive fashion.

$$
P(win|x=1, y) = 0.5 \cdot P(win|x=1, y-1) = 0.5^y 
$$


#### 4.2 Winning probability of player $X$ when $y=1$ and any value of $x$.

Clearly, player $X$ is now in the exact opposite position to before. She only loses if player $Y$ wins all rounds. For example, if $x=3$, player $X$ can occur a maximum of two losses in three games. As a result, the solution is essentially $1$ minus the previous result.

$$
P(win|y=1, x) = 1-P(loss|y=1, x) = 1-0.5^x
$$

Again, this can also be recursively defined as

$$
P(win|y=1, x) = 1-P(loss|y=1, x) = 1 - 0.5\cdot P(loss|y=1, x-1) = 1-0.5^x
$$

#### 4.3 Winning probability of player $X$ for any $x$ and $y$.

I present two solutions to this problem. First, a combinatorial one that is simple and allows to calculate the probabilities directly for any given $x$ and $y$. Second, I also present a solution which is based on representing the game as a Markov Reward Process (MRP).

##### a) Combinatorial Solution
The game can be seen as a combinatorial problem. The maximum number of rounds in the game is $n=x+y-1$. Playing the game is like repeatedly drawing balls (black and white) from an urn with replacement. Let us assume that $black$ means a win for player $X$ while $white$ represents a win for player $Y$. The total number of possible outcomes (where the order does not matter) is therefore $2^{n}$.

The probability of winning this game (from the perspective of player $X$) is therefore the share all such combinations with at least $y$ black balls. The number of combinations with any $k$ black balls is $\binom{n}{k}$. Therefore,

$$
P(win|x,y) = \frac{\text{# of combinations with at least y black balls}} {\text{# of combinations}} = \frac{ \sum_{i=y}^{n} \binom{n}{i} } {2^n}
$$

Also note that $2^n= \sum_{i=0}^{n} \binom{n}{i}$.

In [73]:
from math import comb

def win_probability(x,y):
    
    max_rounds = x+y-1
    total_comb = 2**max_rounds
    win_comb = sum([comb(max_rounds, i) for i in range(y, max_rounds+1)])
    
    print(f'x={x}, y={y}')
    print('max', max_rounds, 'rounds')
    print('# of combinations is', total_comb)
    print('# of winning combinations is', win_comb)
    print(f'P(win | x={x}, y={y}) =', win_comb/total_comb)

In [77]:
x = 6
y = 4

win_probability(x,y)

x=6, y=4
max 9 rounds
# of combinations is 512
# of winning combinations is 382
P(win | x=6, y=4) = 0.74609375


#### b) Markov Reward Process Solution

The game can also be represented as a Markov Reward Process (MRP) with the following characteristics.

The game starts in state $(x,y)$. From there, it proceeds according to the underlying dynamics described in the problem definition. Overall, there are $xy+1$ states (any combination of $x$ and $y$ plus a terminal state). If a win is associated with a reward of $1$ and a loss with $0$, then the resulting state values can be interpreted as the probabilities of winning the game from each state. 

The state value function, or Bellman equation (for an MRP) is 

$$
V(s) = R(s) + \gamma\sum_{s'}P(s'|s)V(s'),\,\text{or}\\\\
V = R+\gamma PV
$$

where the latter is in matrix notation. As already described in **Part 1.1** of the assignment, transforming the Bellman equation leads to a closed form solution of the MRP.

$$V = (I-\gamma P)^{-1}R$$

Essentially, this solution is nothing more than a system of $xy$ linear equations in $xy$ unknowns. Of course, if the state space is too large it can become computationally unfeasible to obtain the closed form solution (which includes the expensive matrix inverse). In this case, we can also fall back to the iterative policy evaluation method described and used in part $1.1$ of this assignment.

Likewise to part 1.1, the solution requires the transition matrix $P$ and the vector of immediate expected rewards $R$. The latter is simply a $xy$ dimensional array with $0.5$ in any state where $y=1$ and $0$ otherwise. The states themselves are represented as a flat vector. That makes the construction of the transition matrix challenging even though conceptually, it is very simple: there is a $0.5$ chance of moving from any state to either of its two "child" or successor states. All other transitions cannot occur. Below is my solution using the starting state $(x=6, y=4)$.

##### First, create and encode all states.

In [78]:
# set x and y
x = 6
y = 4

# create all states
states = np.array(list(itertools.product(range(0,x+1), range(0,y+1))))
n = states.shape[0]

In [79]:
# encode the states
def index_to_state(index):
    return np.unravel_index(index, (n,n))

def state_to_index(state):
    return np.ravel_multi_index(state, (x+1,y+1))

for state in states:
    print(state, 'is encoded as', state_to_index(state))

[0 0] is encoded as 0
[0 1] is encoded as 1
[0 2] is encoded as 2
[0 3] is encoded as 3
[0 4] is encoded as 4
[1 0] is encoded as 5
[1 1] is encoded as 6
[1 2] is encoded as 7
[1 3] is encoded as 8
[1 4] is encoded as 9
[2 0] is encoded as 10
[2 1] is encoded as 11
[2 2] is encoded as 12
[2 3] is encoded as 13
[2 4] is encoded as 14
[3 0] is encoded as 15
[3 1] is encoded as 16
[3 2] is encoded as 17
[3 3] is encoded as 18
[3 4] is encoded as 19
[4 0] is encoded as 20
[4 1] is encoded as 21
[4 2] is encoded as 22
[4 3] is encoded as 23
[4 4] is encoded as 24
[5 0] is encoded as 25
[5 1] is encoded as 26
[5 2] is encoded as 27
[5 3] is encoded as 28
[5 4] is encoded as 29
[6 0] is encoded as 30
[6 1] is encoded as 31
[6 2] is encoded as 32
[6 3] is encoded as 33
[6 4] is encoded as 34


Note that there are several terminal states but can be considered one state.

##### Second, create the transition matrix $P$.

In [80]:
def get_children(state):
    
    child1 = state + np.array([0,-1])
    child2 = state + np.array([-1,0])
    
    children = (child1, child2)

    return [child for child in children if (child>=0).all()]

# for example
# get_children([4,6])

In [81]:
# create the transition matrix
P = np.zeros((n,n))

for state in states:
    
    children = get_children(state)
    
    for child in children:
        state_index = state_to_index(state)
        child_index = state_to_index(child)
        P[state_index, child_index] = 0.5
        
print(P)

[[0.  0.  0.  ... 0.  0.  0. ]
 [0.5 0.  0.  ... 0.  0.  0. ]
 [0.  0.5 0.  ... 0.  0.  0. ]
 ...
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.5 0.  0. ]
 [0.  0.  0.  ... 0.  0.5 0. ]]


##### Third, create the vector of expected immediate rewards $R$

In [82]:
# create vector of expected immediate rewards
R = np.zeros(n)

for i, state in enumerate(states):
    if state[1] == 1 and state[0] != 0:
        R[i] = 0.5
        
print(R)

[0.  0.  0.  0.  0.  0.  0.5 0.  0.  0.  0.  0.5 0.  0.  0.  0.  0.5 0.
 0.  0.  0.  0.5 0.  0.  0.  0.  0.5 0.  0.  0.  0.  0.5 0.  0.  0. ]


##### Finally, solve the MRP.
As can be seen from output below, computing the solution for a single starting state - say $(x=6, y=4)$ - requires evaluating all states of the MRP. This is because the value of one state depends on the value of its successor states (as is the case with dynamic programming). Therefore, this solution is quite expensive computationally.

In [83]:
# solve MDP in closed form
V = np.linalg.inv(np.eye(n)-P).dot(R)

In [84]:
# print out the results
for state, v in zip(states, V):
    if v != 0: # only non-terminal states can be stating states
        print(f'P(win | x={state[0]}, y={state[1]}) =', v)

P(win | x=1, y=1) = 0.5
P(win | x=1, y=2) = 0.25
P(win | x=1, y=3) = 0.125
P(win | x=1, y=4) = 0.0625
P(win | x=2, y=1) = 0.75
P(win | x=2, y=2) = 0.5
P(win | x=2, y=3) = 0.3125
P(win | x=2, y=4) = 0.1875
P(win | x=3, y=1) = 0.875
P(win | x=3, y=2) = 0.6875
P(win | x=3, y=3) = 0.5
P(win | x=3, y=4) = 0.34375
P(win | x=4, y=1) = 0.9375
P(win | x=4, y=2) = 0.8125
P(win | x=4, y=3) = 0.65625
P(win | x=4, y=4) = 0.5
P(win | x=5, y=1) = 0.96875
P(win | x=5, y=2) = 0.890625
P(win | x=5, y=3) = 0.7734375
P(win | x=5, y=4) = 0.63671875
P(win | x=6, y=1) = 0.984375
P(win | x=6, y=2) = 0.9375
P(win | x=6, y=3) = 0.85546875
P(win | x=6, y=4) = 0.74609375
